In [95]:
from scipy.io import loadmat
import os
import numpy as np
import pandas as pd
from imblearn.ensemble import RUSBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
import pickle

In [2]:
Data_path = 'D:\Classifier\Calssification2022_JZ\Code&Dataset\Week13_Classification_Final\Preprocessing'
Data_name = os.listdir(Data_path)
Data_name = [p for p in Data_name if 'mat' in p.split('.')]
Data_name = [p for p in Data_name if ('Site1' not in p.split('_'))&('Site2' not in p.split('_'))]

## 0: veh, 1:ped, 2:bike, 3:wheelchair 

In [42]:
Indicators = {
    'VehData':0,
    'PedData':1,
    'BikeData':2,
    'WheelchairData':3
}
column_names = ['TrajID','Mean_dis','Point_count','Dir','Height','Var_height','Len','Area','Label']
Train_Data = []
for i in range(len(Data_name)):
    data = loadmat(os.path.join(Data_path,Data_name[i]))
    class_name = list(data.keys())[3]
    data = data[class_name]
    class_ind = Indicators[class_name]
    class_col = (np.ones(len(data))*class_ind).reshape(-1,1).astype('int')
    try:
        data = np.concatenate([data[:,-3].reshape(-1,1).astype(int),data[:,:7],class_col],axis = 1)
        data = pd.DataFrame(data,columns=column_names)
    except:
        continue
    Train_Data.append(data)

In [52]:
Valid_train_data = []
for Trajs in Train_Data:
    for ind,traj in Trajs.groupby('TrajID'):
        if ind == -1:
            continue
        if len(traj) > 20:
            traj.Len = traj.Len.max()
            Valid_train_data.append(traj)

In [53]:
Valid_train_data = pd.concat(Valid_train_data)

In [54]:
X,y = np.array(Valid_train_data.iloc[:,1:-1]),np.array(Valid_train_data.iloc[:,-1])

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [56]:
rusboost = RUSBoostClassifier(n_estimators=200, algorithm='SAMME.R',
                              random_state=0)
rusboost.fit(X_train, y_train) 

RUSBoostClassifier(n_estimators=200, random_state=0)

In [57]:
y_pred = rusboost.predict(X_test)

In [58]:
confusion_matrix(y_test, y_pred,normalize = 'true')

array([[9.98806376e-01, 0.00000000e+00, 1.15308563e-03, 4.05381667e-05],
       [2.40905806e-04, 8.38593110e-01, 1.27439171e-01, 3.37268128e-02],
       [8.33614808e-02, 1.24974667e-01, 6.87090455e-01, 1.04573397e-01],
       [6.83060109e-03, 3.63387978e-01, 3.14207650e-01, 3.15573770e-01]])

In [59]:
balanced_accuracy_score(y_test, y_pred)  

0.7100159278566314

# Train Model and save

In [61]:
path = 'E:\Data\Calssification2022_JZ\Code&Dataset\ThreeCasesinPaper\Case 1_TrajectoryGreaterThan10Frames\ClassificationConsideringHistoricalTrajectory\RUSBoost'

In [62]:
X_train = loadmat(os.path.join(path,'Train_Data.mat'))['Train_Data']
y_train = loadmat(os.path.join(path,'Train_Target.mat'))['Train_Target']
y_train = y_train.flatten()
X_test = loadmat(os.path.join(path,'Test_Data.mat'))['Test_Data']
y_test = loadmat(os.path.join(path,'Test_Target.mat'))['Test_Target']
y_test = y_test.flatten()
ind = [i for i in range(7) if i!= 4]
X_train = X_train[:,ind] 
X_test = X_test[:,ind] 

In [63]:
X_test[:,2] = np.zeros(len(X_test))

In [111]:
X_test.shape

(6672, 6)

In [94]:
rusboost = RUSBoostClassifier(n_estimators=2390, algorithm='SAMME.R',
                          random_state=0)
rusboost.fit(X_train, y_train) 

RUSBoostClassifier(n_estimators=2400, random_state=0)

In [98]:
y_pred = rusboost.predict(X_test)

In [99]:
confusion_matrix(y_test, y_pred)

array([[2904,    0,    0,    0],
       [   0, 2659,    0,    3],
       [   1,  157,  188,   44],
       [   0,  111,   68,  537]], dtype=int64)

In [100]:
balanced_accuracy_score(y_test, y_pred)  

0.8077310774624824

In [101]:
filename = 'E:\Data\Veteran\TEST_model.sav'
pickle.dump(rusboost, open(filename, 'wb'))

In [102]:
model = pickle.load(open(filename, 'rb'))

In [103]:
y_pred = model.predict(X_test)

In [104]:
confusion_matrix(y_test, y_pred)

array([[2904,    0,    0,    0],
       [   0, 2659,    0,    3],
       [   1,  157,  188,   44],
       [   0,  111,   68,  537]], dtype=int64)

In [107]:
data = pd.read_csv(r'E:/Data/Veteran/OutputFile/OutputTrajs/Trajctories_0.csv')

In [145]:
data_test = data.loc[:,['ObjectID','Distance_Mea','Point_Cnt','Dir_X_Bbox','Height','Length','Area']]
data_temp = []
for i,df in data_test.groupby('ObjectID'):
    df.Length = df.Length.max()
    data_temp.append(df)
data_temp = pd.concat(data_temp)
data_temp = data_temp.fillna(method = 'pad')

In [146]:
X_test = np.array(data_temp.iloc[:,1:])

In [147]:
y_pred = model.predict(X_test)

In [151]:
pd.concat([data,pd.DataFrame(y_pred.reshape(-1,1),columns=['Class'])],axis = 1)

ObjectID  FrameIndex  Coord_X_Mea  Coord_Y_Mea  Coord_Z_Mea  \
0             2           2    39.217675   -30.201676          0.0   
1             2           3    38.131678   -26.305282          0.0   
2             2           4    38.095683   -29.978346          0.0   
3             2           5    38.115216   -26.293926          0.0   
4             2           6    36.958699   -29.715557          0.0   
...         ...         ...          ...          ...          ...   
27565       299        1976    33.745075    92.713833          0.0   
27566       299        1977    33.239692    91.325304          0.0   
27567       299        1978    33.275824    91.424574          0.0   
27568       299        1979    32.309959    88.770883          0.0   
27569       299        1980    32.304487    88.755847          0.0   

       Distance_Mea  Longitude_Mea  Latitude_Mea  Elevation_Mea  Coord_X_Est  \
0         49.499164    -119.736285     39.484605    1316.037503    39.217675   
1         46.324861    -119.736300     39.484640    1316.062044    38.132721   
2         48.476616    -119.736298     39.484607    1316.099255    38.095687   
3         46.304862    -119.736300     39.484640    1316.062872    38.115167   
4         47.423199    -119.736312     39.484609    1316.161484    36.959752   
...             ...            ...           ...            ...          ...   
27565     98.664000    -119.736427     39.485712    1315.173629    33.746004   
27566     97.186359    -119.736432     39.485699    1315.215674    33.240150   
27567     97.292000    -119.736431     39.485700    1315.212668    33.275744   
27568     94.468000    -119.736441     39.485675    1315.293023    32.310820   
27569     94.452000    -119.736441     39.485675    1315.293478    32.304393   

       ...  Latitude_Est  Elevation_Est  Point_Cnt  Dir_X_Bbox  Dir_Y_Bbox  \
0      ...     39.484605    1316.037503      102.0    0.841271   -0.540614   
1      ...     39.484640    1316.062020      135.0    0.998495   -0.054851   
2      ...     39.484607    1316.099220      115.0    0.989753   -0.142788   
3      ...     39.484640    1316.062909      142.0    0.140813    0.990036   
4      ...     39.484609    1316.161392      120.0    0.984801   -0.173689   
...    ...           ...            ...        ...         ...         ...   
27565  ...     39.485712    1315.173552       47.0    0.342020    0.939693   
27566  ...     39.485699    1315.215636       49.0    0.342021    0.939692   
27567  ...     39.485700    1315.212675       49.0    0.244420    0.969669   
27568  ...     39.485675    1315.292951       49.0    0.342020    0.939692   
27569  ...     39.485675    1315.293486       48.0    0.275527    0.961293   

         Height    Length     Width       Area  Class  
0      1.693463  5.433165  3.386867  18.401407      1  
1      1.684646  4.966858  4.893590  24.305769      1  
2      1.669520  4.980577  1.715877   8.546059      1  
3      1.679414  5.237076  4.974253  26.050539      1  
4      1.636131  4.940226  1.739250   8.592288      1  
...         ...       ...       ...        ...    ...  
27565  3.286137  6.436378  2.449080  15.763207      1  
27566  3.238534  5.008243  2.670043  13.372225      1  
27567  3.188976  6.458226  2.362188  15.255545      1  
27568  3.137046  6.463741  2.563366  16.568932      1  
27569  3.089862  6.600310  2.277019  15.029030      1  

[27570 rows x 29 columns]

In [152]:
def process_traj_data(data):
    data_test = data.loc[:,['ObjectID','Distance_Mea','Point_Cnt','Dir_X_Bbox','Height','Length','Area']]
    data_temp = []
    for i,df in data_test.groupby('ObjectID'):
        df.Length = df.Length.max()
        data_temp.append(df)
    data_temp = pd.concat(data_temp)
    return data_temp

In [ ]:
def get_classified_traj(processed_data):
    